In [36]:
import torch.nn as nn
import torch
from torch.autograd import Variable

In [12]:


class Controller(nn.Module):
    """
    Insert enlightening comment here.
    """
    def __init(self, program_matrix, initial_memory, initial_registers, instruction_register):
        """
        Setup, etc. etc.
        TODO: Check what dimensions program_matrix would be, say that here
        """
        super(Controller, self).__init__()
        R, M = initial_registers.size()
        self.M = M
        self.R = R
        
        # TODO: INSERT DIMENSIONS
        self.program_matrix = program_matrix
        # TODO: What's the difference between Variable and Parameter again?
        
        # Memory matrix (M x M)
        self.memory = initial_memory
        
        # Register Matrix (R x M)
        self.registers = initial_registers
        
        # Instruction Register (M)
        self.IR = instruction_register
        
        #TODO: Add in some fuzz factor????
         
    def forward(self):
        print("HI")
        
        
        
        
        
        
# HOW TO BLUR? ==> Add a constant, then softmax
        
        
        
        
        
        
        
        
        
        
        

NameError: name 'nn' is not defined

In [110]:
class Machine(nn.Module):
    """
    Insert exquisitely eloquent comment here.
    """
    def __init__(self, M, R, stop_threshold):
        """
        Maybe stuff here?  I'm not actually sure what needs to be set up.
        """
        super(Machine, self).__init__()
        self.R = R
        self.M = M # Memory length (also largest number)
        self.N = 3 # Number of instructions
        self.ops = [Jump(M), Stop(M), Add(M)]
        self.jump_index = 0 # TODO: Switch this later
        self.stop_index = 1 # TODO: Switch this later
        self.stop_threshold = stop_threshold
        self.stop_probability = 0
        
        
    def forward(self, e, a, b, o, memory, registers, IR):
        """
        At this point, Mudd CS profs would probably be freaking out about bad variable names.
        But e,a,b,o are what the paper used, so yeah.
        a,b,o = Rx1
        e = Nx1 => N = # instructions
        """
        
        arg1 = torch.matmul(a, registers)
        arg2 = torch.matmul(b, registers)
        
        # Would be kinda nice to not have to for-loop this
        # Or maybe do this once
        # CUUUUUDAAAA
        out = Variable(torch.zeros(self.N, self.M))
        for k in range(self.N):
            for i in range(self.M):
                for j in range(self.M):
                    val = self.ops[k](i,j)
                    out[k, val] = out[k, val] + arg1[i] * arg2[j]
            
        out_vec = torch.matmul(e, out)
        registers = self.write(out_vec, o, registers)
        IR = self.updateIR(e, IR, arg1, arg2)
        should_stop = self.updateStop(e)
        
        return(memory, registers, IR)
        
        
    def write(self, out, o, registers):
        
        ugh = 1-o
        ugh = torch.unsqueeze((1-o),1)
        ugh = ugh.expand(self.R, self.M)
        ugh = ugh * registers
        registers = torch.matmul(torch.unsqueeze(o,1), torch.unsqueeze(out,0)) + ugh
        return registers
    
    def updateIR(self, e, IR, arg1, arg2):
        # IR - length M vector
        jump_probability = e[self.jump_index]
        is_zero = arg1[0]
        # If we're not jumping, just shift IR by one slot
        IR_no_jump = torch.cat([IR[-1], IR[:-1]])
        # If we are on a jump instruction, check whether the argument's 0.
        # If it is, jump to the location specified by arg2.  Otherwise, increment like normal.
        IR_jump = arg2 * is_zero + (1 - is_zero) * IR_no_jump
        
        IR = IR_no_jump * (1 - jump_probability) + IR_jump * jump_probability
        return IR
        
    def updateStop(self, e):
        self.stop_probability += e[self.stop_index].data[0]
        print("PROB", self.stop_probability)
        if self.stop_probability > self.stop_threshold:
            return True #should stop
        return False #shouldn't stop
    
    
    
    #TODO: Add write operation!
        
        


In [4]:
class Operation(nn.Module):
    """
    COOOOOOOMMMMMMMENNNNNT.  This is what happens when I code at 2 a.m.
    """
    def __init__(self, M):
        """
        Do the things that neeeeeeeed to work.
        """
        super(Operation, self).__init__()
        self.M = M
    
    def forward(self, x, y):
        raise NotImplementedError
        
#     def makeOneHotVec(self, num):
#         new_vec = nn.Variable(torch.zeros(self.M))
#         new_vec[num] = 1
#         return new_vec

In [5]:
class Add(Operation):
    """
    x+y
    """
    def __init__(self, M):
        super(Add, self).__init__(M)
    
    def forward(self, x, y):
        return (x + y) % self.M

In [6]:
class Stop(Operation):
    
    def __init__(self, M):
        super(Stop, self).__init__(M)

    def forward(self, x, y):
        return 0

In [7]:
class Jump(Operation):
    
    def __init__(self, M):
        super(Jump, self).__init__(M)

    def forward(self, x, y):
        return 0

In [8]:
machine = Machine(4, 2, .5)
e = Variable(torch.FloatTensor([.3,.3,.4]))
a = Variable(torch.FloatTensor([.1, .9]))
b = Variable(torch.FloatTensor([.8, .2]))
o = Variable(torch.FloatTensor([.6, .4]))
memory = Variable(torch.FloatTensor([[1,0,0,0], [1,1,0,0], [1,0,1,0]]))
registers = Variable(torch.FloatTensor([[.4, .5, 0, .1], [.2, 0, .6, .2]]))
IR = Variable(torch.FloatTensor([.1, .9, .1, .1]))
mem, regs, ir = machine(e, a, b, o, memory, registers, IR)
print("MEM", mem)
print("REGS", regs)
print("IR", ir)

PROB 0.30000001192092896
MEM Variable containing:
 1  0  0  0
 1  1  0  0
 1  0  1  0
[torch.FloatTensor of size 3x4]

REGS Variable containing:
 0.5742  0.2465  0.0633  0.1160
 0.3962  0.0310  0.4022  0.1707
[torch.FloatTensor of size 2x4]

IR Variable containing:
 0.1172
 0.1198
 0.8485
 0.1013
[torch.FloatTensor of size 4]



In [9]:
# TASKS
# - Compilation
# - Rest of the operations
# - Train function
# - Controller
# - Cuda
# - Blurring
# - Running the tests they ran, verifying that we get similar results

In [10]:
#PUT THE TRAINING FUNCTION HERE.  COPY AND PASTE A BUNCHA STUFF